In [ ]:
!pip install deepeval

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 708.6/708.6 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.3/132.3 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.9/65.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.0/208.0 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 357.5/357.5 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46.4 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.6/40.6 kB 2.2 MB/s eta 0:00:00
  Attempting uninstall: opentelemetry-proto
    Found existing installation: opentelemetry-proto 1.37.0
    Uninstalling opentele

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = "YOUR-API-KEY"

Want to use other evaluation models? [Click here](https://deepeval.com/integrations/models/openai) to see all supported models and their usage instructions.

In [ ]:
from deepeval.test_case import ConversationalTestCase, Turn

test_case = ConversationalTestCase(
    turns=[
        Turn(role="user", content="what's the weather like today?"),
        Turn(role="assistant", content="Where do you live bro? T~T"),
        Turn(role="user", content="Just tell me the weather in Paris"),
        Turn(
            role="assistant",
            content="The weather in Paris today is sunny and 24°C.",
        ),
        Turn(role="user", content="Should I take an umbrella?"),
        Turn(
            role="assistant",
            content="You trying to be stylish? I don't recommend it.",
        ),
    ],
    scenario="User asks about weather",
    expected_outcome="Assistant provides weather info in an angry tone.",
)

In [ ]:
from deepeval.metrics.dag import DeepAcyclicGraph
from deepeval.metrics.conversational_dag import (
    ConversationalTaskNode,
    ConversationalBinaryJudgementNode,
    ConversationalNonBinaryJudgementNode,
    ConversationalVerdictNode,
)
from deepeval.test_case import TurnParams

non_binary_node = ConversationalNonBinaryJudgementNode(
    criteria="How was the assistant's behaviour towards user?",
    evaluation_params=[TurnParams.ROLE, TurnParams.CONTENT],
    children=[
        ConversationalVerdictNode(verdict="Rude", score=0),
        ConversationalVerdictNode(verdict="Neutral", score=5),
        ConversationalVerdictNode(verdict="Nice", score=10),
    ],
)

binary_node = ConversationalBinaryJudgementNode(
    criteria="Do the assistant's replies satisfy user's questions?",
    children=[
        ConversationalVerdictNode(verdict=False, score=0),
        ConversationalVerdictNode(verdict=True, child=non_binary_node),
    ],
)

task_node = ConversationalTaskNode(
    instructions="Summarize the conversation and explain assiatant's behaviour overall.",
    output_label="Summary",
    evaluation_params=[TurnParams.ROLE, TurnParams.CONTENT],
    children=[binary_node],
)


dag = DeepAcyclicGraph(root_nodes=[task_node])

ConversationalTaskNode(instructions="Summarize the conversation and explain assiatant's behaviour overall.", output_label='Summary', children=[ConversationalBinaryJudgementNode(criteria="Do the assistant's replies satisfy user's questions?", children=[ConversationalVerdictNode(verdict=False, score=0, child=None, _parent=...), ConversationalVerdictNode(verdict=True, score=None, child=ConversationalNonBinaryJudgementNode(criteria="How was the assistant's behaviour towards user?", children=[ConversationalVerdictNode(verdict='Rude', score=0, child=None, _parent=...), ConversationalVerdictNode(verdict='Neutral', score=5, child=None, _parent=...), ConversationalVerdictNode(verdict='Nice', score=10, child=None, _parent=...)], evaluation_params=[<TurnParams.ROLE: 'role'>, <TurnParams.CONTENT: 'content'>], turn_window=None, label=None, _verbose_logs=None, _verdict=None, _parents=None), _parent=...)], evaluation_params=None, turn_window=None, label=None, _verbose_logs=None, _verdict=None, _paren

In [ ]:
from deepeval.metrics import ConversationalDAGMetric

playful_chatbot_metric = ConversationalDAGMetric(
    name="Playful Chatbot",
    dag=dag,
)

In [ ]:
from deepeval import evaluate

evaluate([test_case], [playful_chatbot_metric])

✨ You're running DeepEval's latest Playful Chatbot [ConversationalDAG] Metric! (using gpt-4.1, strict=False, 
async_mode=True)...

Output()



Metrics Summary

  - ❌ Playful Chatbot [ConversationalDAG] (score: 0.0, threshold: 0.5, strict: False, evaluation model: gpt-4.1, reason: The score is 0.0 because, although the assistant answered the user's questions, its behavior was judged as rude due to the use of overly informal and dismissive language (e.g., 'Where do you live bro? T~T' and 'You trying to be stylish? I don't recommend it.'). This tone was interpreted as mocking and not taking the user's questions seriously, which is inappropriate for a playful chatbot and led to the lowest possible score., error: None)

For conversational test case:

  Turns:
 0. user      what's the weather like today?
 1. assistant Where do you live bro? T~T
 2. user      Just tell me the weather in Paris
 3. assistant The weather in Paris today is sunny and 24°C.
 4. user      Should I take an umbrella?
 5. assistant You trying to be stylish? I don't recommend it.


Overall Metric Pass Rates

Playful Chatbot [ConversationalDAG]: 0.00% pass ra

⚠ WARNING: No hyperparameters logged.
» ]8;id=692746;https://deepeval.com/docs/evaluation-prompts\Log hyperparameters]8;;\ to attribute prompts and models to your test runs.

================================================================================

✓ Evaluation completed 🎉! (time taken: 8.83s | token cost: 0.005575999999999999 USD)
» Test Results (1 total tests):
   » Pass Rate: 0.0% | Passed: 0 | Failed: 1

 ================================================================================ 

» Want to share evals with your team, or a place for your test cases to live? ❤️ 🏡
  » Run 'deepeval view' to analyze and save testing results on Confident AI.

EvaluationResult(test_results=[TestResult(name='conversational_test_case_0', success=False, metrics_data=[MetricData(name='Playful Chatbot [ConversationalDAG]', threshold=0.5, success=False, score=0.0, reason="The score is 0.0 because, although the assistant answered the user's questions, its behavior was judged as rude due to the use of overly informal and dismissive language (e.g., 'Where do you live bro? T~T' and 'You trying to be stylish? I don't recommend it.'). This tone was interpreted as mocking and not taking the user's questions seriously, which is inappropriate for a playful chatbot and led to the lowest possible score.", strict_mode=False, evaluation_model='gpt-4.1', error=None, evaluation_cost=0.005575999999999999, verbose_logs="______________________________________________\n| ConversationalTaskNode | Level == 0 |\n**********************************************\nLabel: None\n\nInstructions:\nSummarize the conversation and explain assiatant's behaviour overall.\n\nSummary:

# Task
Create an AI agent that uses the Sonar API to return the latest technology trends.

## Install necessary libraries

### Subtask:
Install libraries required for making API calls and potentially for processing the response (e.g., `requests`).


**Reasoning**:
The subtask is to install the `requests` library. The `!pip install` command is used to install Python packages in a Jupyter notebook environment.



In [ ]:
!pip install requests

In [ ]:
# This is a placeholder cell for the markdown explanation.
# The actual explanation will be provided in a markdown cell.
pass

**Reasoning**:
I will now provide the explanation on how to obtain a Sonar API key and securely store it using Colab secrets in a markdown cell.



In [ ]:
%%markdown
## Obtaining and Securely Storing Your Sonar API Key
"""
To access the Sonar API, you'll first need an API key. Follow these steps:

1.  **Visit the Sonar Website:** Go to the official Sonar website at [https://www.getsonar.com/](https://www.getsonar.com/).
2.  **Navigate to API/Developer Section:** Look for sections like "API Documentation," "Developer," "Settings," or "Account." The exact location may vary, but you should find information on accessing their API.
3.  **Generate an API Key:** Within the API or developer section, there should be an option to generate a new API key. Follow their specific instructions to create one. This key is unique to your account and is essential for making authenticated requests to the Sonar API.

**Securely Storing Your API Key using Colab Secrets**

It is crucial to protect your API key and avoid exposing it directly in your code. Google Colab provides a secure way to store sensitive information like API keys using "Secrets."

1.  **Access Colab Secrets:** In your Google Colab notebook, find the "🔑" icon in the left sidebar. Click on it to open the Secrets panel.
2.  **Add a New Secret:** Click on the "Add new secret" button.
3.  **Name the Secret:** In the "Name" field, enter a descriptive name for your secret, such as `SONAR_API_KEY`. This name will be used to access the key in your code.
4.  **Paste the API Key:** In the "Value" field, paste the Sonar API key you obtained from the Sonar website.
5.  **Save the Secret:** Click "Save secret."

**Accessing the Stored Secret in Your Code**

Once your secret is stored, you can access it in your Colab notebook using the `os.environ` dictionary. Make sure you have imported the `os` module.

```python
import os
from google.colab import userdata

# Access the stored API key
sonar_api_key = userdata.get('SONAR_API_KEY')

# You can now use sonar_api_key in your API calls
# print(sonar_api_key) # Avoid printing your API key in output
```

By using Colab secrets, you keep your API key secure and separate from your code, making your notebook safer to share. Remember to replace `'SONAR_API_KEY'` with the exact name you gave your secret if you chose a different one.


## Obtaining and Securely Storing Your Sonar API Key
"""
To access the Sonar API, you'll first need an API key. Follow these steps:

1.  **Visit the Sonar Website:** Go to the official Sonar website at [https://www.getsonar.com/](https://www.getsonar.com/).
2.  **Navigate to API/Developer Section:** Look for sections like "API Documentation," "Developer," "Settings," or "Account." The exact location may vary, but you should find information on accessing their API.
3.  **Generate an API Key:** Within the API or developer section, there should be an option to generate a new API key. Follow their specific instructions to create one. This key is unique to your account and is essential for making authenticated requests to the Sonar API.

**Securely Storing Your API Key using Colab Secrets**

It is crucial to protect your API key and avoid exposing it directly in your code. Google Colab provides a secure way to store sensitive information like API keys using "Secrets."

1.  **Access Colab Secrets:** In your Google Colab notebook, find the "🔑" icon in the left sidebar. Click on it to open the Secrets panel.
2.  **Add a New Secret:** Click on the "Add new secret" button.
3.  **Name the Secret:** In the "Name" field, enter a descriptive name for your secret, such as `SONAR_API_KEY`. This name will be used to access the key in your code.
4.  **Paste the API Key:** In the "Value" field, paste the Sonar API key you obtained from the Sonar website.
5.  **Save the Secret:** Click "Save secret."

**Accessing the Stored Secret in Your Code**

Once your secret is stored, you can access it in your Colab notebook using the `os.environ` dictionary. Make sure you have imported the `os` module.

```python
import os
from google.colab import userdata

# Access the stored API key
sonar_api_key = userdata.get('SONAR_API_KEY')

# You can now use sonar_api_key in your API calls
# print(sonar_api_key) # Avoid printing your API key in output
```

By using Colab secrets, you keep your API key secure and separate from your code, making your notebook safer to share. Remember to replace `'SONAR_API_KEY'` with the exact name you gave your secret if you chose a different one.


## Define a function to interact with the sonar api

### Subtask:
Create a Python function that takes a query as input and uses the Sonar API to fetch technology trends.


**Reasoning**:
Define the Python function `get_technology_trends` to fetch data from the Sonar API, incorporating steps to construct the URL, prepare headers with the API key retrieved from Colab secrets, make the API request, handle the response, and include basic error handling.



In [ ]:
import requests
from google.colab import userdata

def get_technology_trends(query):
    """
    Fetches technology trends from the Sonar API based on a query.

    Args:
        query: The technology query string.

    Returns:
        A dictionary containing the technology trends data if successful,
        otherwise None.
    """
    # Sonar API endpoint (this is a placeholder, refer to Sonar API docs)
    # Replace with the actual API endpoint for technology trends
    api_url = "https://api.getsonar.com/v1/trends"

    # Retrieve API key securely from Colab secrets
    sonar_api_key = "YOUR-API-KEY"

    if not sonar_api_key:
        print("Error: Sonar API key not found in Colab secrets.")
        return None

    # Prepare headers with authorization
    headers = {
        "Authorization": f"Bearer {sonar_api_key}",
        "Content-Type": "application/json" # Adjust content type if needed
    }

    # Prepare query parameters (adjust based on API documentation)
    params = {
        "q": query
    }

    try:
        # Make the GET request to the Sonar API
        response = requests.get(api_url, headers=headers, params=params)

        # Check if the request was successful
        response.raise_for_status()  # Raise an exception for bad status codes

        # Parse the JSON response
        trends_data = response.json()

        return trends_data

    except requests.exceptions.RequestException as e:
        print(f"Error during Sonar API request: {e}")
        return None
    except ValueError as e:
        print(f"Error parsing JSON response: {e}")
        return None


## Implement the agent's logic

### Subtask:
Implement the core logic within the AI agent class to process user input, extract the technology trend query using the language model, call the `get_technology_trends` function, and format the response for the user.


**Reasoning**:
Implement the core logic within the AI agent class to process user input, extract the technology trend query using the language model, call the `get_technology_trends` function, and format the response for the user, including the necessary error handling and data parsing.



In [ ]:
import openai
# Assuming get_technology_trends is defined elsewhere and accessible
# from .api_functions import get_technology_trends # Example if in another file

class TechnologyTrendAgent:
    """
    An AI agent that interacts with a language model to understand user queries about technology trends and fetches data using the Sonar API.
    """
    def __init__(self):
        """
        Initializes the AI agent with the OpenAI API key.

        Args:
            api_key: The OpenAI API key for the language model.
        """
        openai.api_key = "YOUR-API-KEY"
        self.conversation_history = [] # Optional: to maintain conversation context

    def process_query(self, user_input):
        """
        Processes user input, extracts the technology trend query, calls the
        Sonar API function, and returns the response.

        Args:
            user_input: The user's input string.

        Returns:
            A string containing the technology trends data or an error message.
        """
        self.conversation_history.append({"role": "user", "content": user_input})

        try:
            # Use the language model to understand the user's query and extract it
            # Using function calling for more structured extraction
            response = openai.chat.completions.create(
                model="gpt-4o-mini", # Or another suitable model that supports function calling
                messages=[
                    {"role": "system", "content": "You are a helpful assistant that identifies technology trend queries from user input."},
                    {"role": "user", "content": f"Extract the technology trend query from the following text: '{user_input}'"}
                ],
                tools=[
                    {
                        "type": "function",
                        "function": {
                            "name": "get_technology_trends",
                            "description": "Get the latest technology trends for a given query.",
                            "parameters": {
                                "type": "object",
                                "properties": {
                                    "query": {
                                        "type": "string",
                                        "description": "The technology trend query (e.g., 'artificial intelligence', 'blockchain')."
                                    }
                                },
                                "required": ["query"]
                            }
                        }
                    }
                ],
                tool_choice={"type": "function", "function": {"name": "get_technology_trends"}}
            )

            # Extract the function call arguments
            tool_calls = response.choices[0].message.tool_calls
            if tool_calls and tool_calls[0].function.name == "get_technology_trends":
                import json
                try:
                    function_args = json.loads(tool_calls[0].function.arguments)
                    extracted_query = function_args.get("query")
                    print(f"Extracted query using function calling: {extracted_query}") # For debugging
                except json.JSONDecodeError:
                    error_message = "Could not parse function call arguments."
                    self.conversation_history.append({"role": "assistant", "content": error_message})
                    return error_message
            else:
                # Fallback or handle cases where function calling doesn't extract the query
                # For simplicity, let's assume the model directly returns the query if function calling fails
                extracted_query = response.choices[0].message.content.strip()
                print(f"Extracted query using direct response: {extracted_query}") # For debugging


            if not extracted_query:
                 error_message = "Could not extract a technology trend query from your request."
                 self.conversation_history.append({"role": "assistant", "content": error_message})
                 return error_message


            # Call the Sonar API function with the extracted query
            trends_data = get_technology_trends(extracted_query)

            if trends_data:
                # Process and format the trends_data for the user
                formatted_response = f"Here are the technology trends for '{extracted_query}':\n"
                # Assuming trends_data is a dictionary with a 'trends' key containing a list of trend dictionaries
                if isinstance(trends_data, dict) and 'trends' in trends_data and isinstance(trends_data['trends'], list):
                    if trends_data['trends']:
                        for trend in trends_data['trends']:
                            # Adjust key names based on actual Sonar API response structure
                            name = trend.get('name', 'N/A')
                            description = trend.get('description', 'No description available.')
                            formatted_response += f"- **{name}**: {description}\n"
                    else:
                        formatted_response += "No specific trends found."
                else:
                    formatted_response += "Could not retrieve detailed trend information or the data format was unexpected."


                self.conversation_history.append({"role": "assistant", "content": formatted_response})
                return formatted_response
            else:
                error_message = f"Could not retrieve technology trends for '{extracted_query}'. The Sonar API might have returned an error or no data."
                self.conversation_history.append({"role": "assistant", "content": error_message})
                return error_message

        except Exception as e:
            error_message = f"An error occurred while processing your request: {e}"
            self.conversation_history.append({"role": "assistant", "content": error_message})
            return error_message




## Test the agent

### Subtask:
Write test cases to ensure the agent correctly retrieves and presents technology trends.


**Reasoning**:
Write test cases for the TechnologyTrendAgent using unittest and mocking.



In [ ]:
agent = TechnologyTrendAgent()
output = agent.process_query("What is the most common ai agent framework right now?")

print(output)



Extracted query using function calling: ai agent framework
Error during Sonar API request: HTTPSConnectionPool(host='api.getsonar.com', port=443): Max retries exceeded with url: /v1/trends?q=ai+agent+framework (Caused by SSLError(SSLError(1, '[SSL: TLSV1_UNRECOGNIZED_NAME] tlsv1 unrecognized name (_ssl.c:1010)')))
Could not retrieve technology trends for 'ai agent framework'. The Sonar API might have returned an error or no data.
